In [1]:
import pandas as pd
import re

In [2]:
df=pd.read_csv("test.csv")

In [3]:
df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
0,655269,53478,63558,86.54,67,22807,45,1497,01-01-1974,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
1,723482,55513,63163,89.45,67,22807,45,1497,20-05-1985,Self employed,...,0,0,0,5605,0,1,0,0yrs 8mon,1yrs 0mon,1
2,758529,65282,84320,79.93,78,23135,86,2071,14-10-1995,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
3,763449,46905,63896,76.58,78,17014,45,2070,01-06-1973,Self employed,...,0,0,0,0,0,0,0,2yrs 5mon,2yrs 5mon,0
4,708663,51428,63896,86.08,78,17014,45,2069,01-06-1972,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0


In [4]:
df['Employment.Type']=df['Employment.Type'].fillna("NVAL")
df['Employment.Type']=df['Employment.Type'].astype("category")

In [5]:
df['Employment.Type']= df['Employment.Type'].cat.codes


In [6]:
for col in ['UniqueID','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID',
           'Employee_code_ID','MobileNo_Avl_Flag','Aadhar_flag','PAN_flag','VoterID_flag','Driving_flag','Passport_flag']:
    df[col] = df[col].astype('category')

In [7]:
df['DisbursalDate']= pd.to_datetime(df['DisbursalDate'])
df['Date.of.Birth']= pd.to_datetime(df['Date.of.Birth'])

In [8]:
import datetime
def from_dob_to_age(born):
    today = datetime.date.today()
    return abs(today.year - born.year - ((today.month, today.day) < (born.month, born.day)))
df['Date.of.Birth']=df['Date.of.Birth'].apply(lambda x: from_dob_to_age(x))
df['DisbursalDate']=df['DisbursalDate'].apply(lambda x: from_dob_to_age(x))

In [9]:
df=df.rename(columns={"Date.of.Birth": "Age.of.Customer", "DisbursalDate": "Disbursal.Age"})

In [10]:
def toMonths(str):
    cache = []
    for k in df[str]:
        temp = int(re.split("[yrs mon]+", k)[0]) * 12 + int(re.split("[yrs mon]+", k)[1])
        cache.append(temp)
    return cache

In [11]:
df['CREDIT.HISTORY.LENGTH'] = toMonths('CREDIT.HISTORY.LENGTH')


In [12]:
df['AVERAGE.ACCT.AGE'] = toMonths('AVERAGE.ACCT.AGE')

In [13]:
df['PERFORM_CNS.SCORE.DESCRIPTION'].replace({"C-Very Low Risk": 0, "A-Very Low Risk": 0,"D-Very Low Risk":0,"B-Very Low Risk":0}, inplace=True)
# Low Risk
df['PERFORM_CNS.SCORE.DESCRIPTION'].replace({"F-Low Risk": 1, "E-Low Risk": 1,"G-Low Risk":1}, inplace=True)
# medium risk
df['PERFORM_CNS.SCORE.DESCRIPTION'].replace({"H-Medium Risk": 2, "I-Medium Risk": 2}, inplace=True)
# High risk
df['PERFORM_CNS.SCORE.DESCRIPTION'].replace({"J-High Risk": 3, "K-High Risk": 3}, inplace=True)
#Very high risk
df['PERFORM_CNS.SCORE.DESCRIPTION'].replace({"L-Very High Risk": 4, "M-Very High Risk": 4}, inplace=True)
# Not scored
df['PERFORM_CNS.SCORE.DESCRIPTION'].replace({"No Bureau History Available": 5, 
                                             "Not Scored: Sufficient History Not Available": 5,
                                             "Not Scored: Not Enough Info available on the customer":5,
                                             "Not Scored: No Activity seen on the customer (Inactive)":5,
                                             "Not Scored: No Updates available in last 36 months":5,
                                             "Not Scored: Only a Guarantor":5,
                                             "Not Scored: More than 50 active Accounts found":5}, inplace=True)
# finally converting the column to category
df['PERFORM_CNS.SCORE.DESCRIPTION'] = df['PERFORM_CNS.SCORE.DESCRIPTION'].astype('category')

In [14]:
df=df.drop(['supplier_id','Current_pincode_ID','Employee_code_ID','State_ID','MobileNo_Avl_Flag'],axis=1)


In [15]:
df.columns


Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'manufacturer_id', 'Age.of.Customer', 'Employment.Type',
       'Disbursal.Age', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES'],
      dtype='object')

In [16]:
df=df.drop(['UniqueID','PRI.ACTIVE.ACCTS','SEC.ACTIVE.ACCTS','SEC.CURRENT.BALANCE','AVERAGE.ACCT.AGE'],axis=1)

In [17]:

df['branch_id']= df['branch_id'].cat.codes
df['manufacturer_id']= df['manufacturer_id'].cat.codes
df['Aadhar_flag']= df['Aadhar_flag'].cat.codes
df['PAN_flag']= df['PAN_flag'].cat.codes
df['VoterID_flag']= df['VoterID_flag'].cat.codes
df['Driving_flag']= df['Driving_flag'].cat.codes
df['Passport_flag']= df['Passport_flag'].cat.codes
df['PERFORM_CNS.SCORE.DESCRIPTION']= df['PERFORM_CNS.SCORE.DESCRIPTION'].cat.codes


In [18]:
df.shape

(112392, 30)

In [20]:
df.to_csv('test_preprocessed.csv')

In [21]:
df=pd.read_csv("test_preprocessed.csv")

In [22]:
df.head()

,Unnamed: 0,disbursed_amount,asset_cost,ltv,branch_id,manufacturer_id,Age.of.Customer,Employment.Type,Disbursal.Age,Aadhar_flag,...,SEC.NO.OF.ACCTS,SEC.OVERDUE.ACCTS,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
0,0,53478,63558,86.54,30,0,47,1,2,1,...,0,0,0,0,0,0,0,0,0,0
1,1,55513,63163,89.45,30,0,35,2,2,1,...,0,0,0,0,5605,0,1,0,12,1
2,2,65282,84320,79.93,39,5,25,1,2,1,...,0,0,0,0,0,0,0,0,0,0
3,3,46905,63896,76.58,39,0,48,2,2,1,...,0,0,0,0,0,0,0,0,29,0
4,4,51428,63896,86.08,39,0,49,1,2,1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df.isnull().sum()

Unnamed: 0                             0
disbursed_amount                       0
asset_cost                             0
ltv                                    0
branch_id                              0
manufacturer_id                        0
Age.of.Customer                        0
Employment.Type                        0
Disbursal.Age                          0
Aadhar_flag                            0
PAN_flag                               0
VoterID_flag                           0
Driving_flag                           0
Passport_flag                          0
PERFORM_CNS.SCORE                      0
PERFORM_CNS.SCORE.DESCRIPTION          0
PRI.NO.OF.ACCTS                        0
PRI.OVERDUE.ACCTS                      0
PRI.CURRENT.BALANCE                    0
PRI.SANCTIONED.AMOUNT                  0
PRI.DISBURSED.AMOUNT                   0
SEC.NO.OF.ACCTS                        0
SEC.OVERDUE.ACCTS                      0
SEC.SANCTIONED.AMOUNT                  0
SEC.DISBURSED.AM

Hence there are no null values present 